In [23]:
#IPython is what you are using now to run the notebook
import IPython
# Pandas makes working with data tables easier
import pandas as pd
import matplotlib.pyplot as plt

In [4]:
# loading data into data frame
data_topCast = pd.read_excel("topCast_label.xlsx")
data_all = pd.read_excel("output_all.xlsx")

In [5]:
data_topCast.head()

,id,season,episode,scene,line_text,speaker,sentiment score,sentiment label
0,1,1,1,1,All right Jim. Your quarterlies look very good...,Michael,0.2,slightly positive
1,2,1,1,1,"Oh, I told you. I couldn't close it. So...",Jim,0.0,neutral
2,3,1,1,1,So you've come to the master for guidance? Is ...,Michael,0.0,neutral
3,4,1,1,1,"Actually, you called me in here, but yeah.",Jim,0.2,slightly positive
4,5,1,1,1,"All right. Well, let me show you how it's done.",Michael,0.1,slightly positive


In [25]:
# fetch data of Character
char_data = data_topCast[data_topCast["speaker"] == "Angela"]
# fetch sentiment of Character
char_sentiment = char_data[char_data["sentiment label"] == "neutral"]


In [26]:
char_data.head()

,id,season,episode,scene,line_text,speaker,sentiment score,sentiment label
49,68,1,1,14,I bet it's gonna be me. Probably gonna be me.,Angela,0.0,neutral
206,246,1,1,46,"My name is Angela and um, I'm in charge of the...",Angela,0.0,neutral
208,248,1,1,46,"Oscar, how do I describe him. He's like a stap...",Angela,0.1,slightly positive
211,251,1,1,46,Not very worried about downsizing right now. B...,Angela,0.0,neutral
213,253,1,1,46,I have a lot of cat figurines on my desk. I th...,Angela,0.1,slightly positive
